# Overview

This example shows how to use `flux-data-qaqc` with a custom climate input file that is not from FLUXNET and generally covers how to set up the config file. The major differences when using `flux-data-qaqc` for different datasets lie in the config file declarations therefore the entire workflow from the [FLUXNET 2015 example notebook](https://github.com/Open-ET/flux-data-qaqc/blob/master/examples/FLUXNET_2015_example.ipynb) will work just the same once your config file is set up correctly. That notebook is recommended to be viewed for all general use whereas this one puts more focus on the formating rules of input data itself. 

---
The data used herein is provided with the software package and can be downloaded [here](https://github.com/Open-ET/flux-data-qaqc/blob/master/examples/), it happens to be from a USGS eddy covariance flux tower for Dixie Valey Dense Vegetation. Details on the data can be found in this [report](https://pubs.usgs.gov/pp/1805/pdf/pp1805.pdf).

In [1]:
%load_ext autoreload
%autoreload 2
from fluxdataqaqc import Data, QaQc, Plot
from bokeh.plotting import figure, show
from bokeh.models.formatters import DatetimeTickFormatter
from bokeh.models import LinearAxis, Range1d
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# Seting up a config file 
---
The config file needed for using `flux-data-qaqc` has two major sections:
1. METADATA
2. DATA

In **METADATA** the "climate_file_path" is the full or relative file path of the input climate file (excel or CSV, more on formating this below) containing the climatic data to be analyzed. The "station_elevation" (expected in meters) and latitude (decimal degrees) fields are used to calculate clear sky potential solar radiation using an ASCE formulation. The item "missing_data_value" is used to correctly parse missing data in the climate time series. "site_id" is used for saving output time series and plot files, and the "qc_threshold" and "qc_flag" items are used for filtering out bad data based on provided quality control data that is included with the input climate file (if they exist).

The **DATA** section of the config file is where you specify climate variables and their units. There are two major functionalities in `flux-data-qaqc`, first, correcting surface energy balance by adjusting latent energy and sensible heat fluxes and calculate other climatic variables. Second, it serves as a robust way to read in different time series data and produce visualizations, e.g. their daily and monthly time series. 

Here is a list of all the "expected" climate variable names in the **DATA** section (the names before the colons are the items in the config file and the names after are their internal names used by `flux-data-qaqc`):

```Python
'datestring_col' : 'date' ,
'year_col' : 'year' ,
'month_col' : 'month' ,
'day_col' : 'day' ,
'net_radiation_col' : 'Rn' ,
'ground_flux_col' : 'G' ,
'latent_heat_flux_col' : 'LE' ,
'latent_heat_flux_corrected_col' : 'LE_user_corr' ,
'sensible_heat_flux_col' : 'H' ,
'sensible_heat_flux_corrected_col' : 'H_user_corr' ,
'shortwave_in_col' : 'sw_in' ,
'shortwave_out_col' : 'sw_out' ,
'shortwave_pot_col' : 'sw_pot' ,
'longwave_in_col' : 'lw_in' ,
'longwave_out_col' : 'lw_out' ,
'vap_press_col' : 'vp' ,
'vap_press_def_col' : 'vpd' ,
'avg_temp_col' : 't_avg' ,
'precip_col' : 'ppt' ,
'wind_spd_col' : 'ws' 
```

You may view these climate entry keys (from the config file) from within Python using the `Data` object's `config` attribute:

In [2]:
config_path = 'USGS_config.ini'
d = Data(config_path)
for each in d.config.items('DATA'):
    print(each[0])

datestring_col
year_col
month_col
day_col
net_radiation_col
net_radiation_units
ground_flux_col
ground_flux_units
latent_heat_flux_col
latent_heat_flux_units
latent_heat_flux_corrected_col
latent_heat_flux_corrected_units
sensible_heat_flux_col
sensible_heat_flux_units
sensible_heat_flux_corrected_col
sensible_heat_flux_corrected_units
shortwave_in_col
shortwave_in_units
shortwave_out_col
shortwave_out_units
shortwave_pot_col
shortwave_pot_units
longwave_in_col
longwave_in_units
longwave_out_col
longwave_out_units
vap_press_col
vap_press_units
vap_press_def_col
vap_press_def_units
avg_temp_col
avg_temp_units
precip_col
precip_units
wind_spd_col
wind_spd_units


# Dealing with missing data

You may not have any of the expected climate variables in your data, if this is the case you may simply specify them as missing ('na') in your config file. Missing variables will be ignored for the most part and not present in output files/plots, however if key variables for the energy balance are not present (LE, H, G, and Rn) then you will not be able to run energy balance closure correction routines. 

# Formating rules for input climate time series files

Generally `flux-data-qaqc` accepts Excel files (.xlx and .xlsx) and comma separated value (CSV) formatted text files. Second, the file should have a column with combined date and time. Currently there is no restriction on the temporal frequency of input data however it is automatically resampled to daily frequency before running correction routines. Lastly, there should be a single header row containing all variable names followed by the first entry of climatic variables. 

Here is an example of a valid input file's first 5 rows and 8 columns:

| date       | t_avg  | sw_pot  | sw_in   | lw_in   | vpd   | ppt | ws    |
|------------|--------|---------|---------|---------|-------|-----|-------|
| 2009-01-01 | 2.803  | 186.71  | 123.108 | 261.302 | 1.919 | 0   | 3.143 |
| 2009-01-02 | 2.518  | 187.329 | 121.842 | 268.946 | 0.992 | 0   | 2.093 |
| 2009-01-03 | 5.518  | 188.008 | 124.241 | 268.004 | 2.795 | 0   | 4.403 |
| 2009-01-04 | -3.753 | 188.742 | 113.793 | 246.675 | 0.892 | 0   | 4.336 |

---
# Creating your own quality control values or flags to filter out bad data

Currently `flux-data-qaqc` supports filtering out poor quality data based on quality control (QC) values (numeric) or flags (characters) that exist within the input data using the `Data.apply_qc_flags` method. 

Let's say that you have a column in your input data named 'QC_flag' that contains character strings signifying the quality of data for the given date-time, if this flag is 'g' the corresponding data point is 'good' and if the flag is 'b' the data point is bad quality and you would like to ignore it. Further lets say that you want this to apply to only your LE and H columns, then in your config file you should declare the flag in the **METADATA** section:

```bash
qc_flag = b
```

and in the **DATA** section of your config you will state that the 'QC_flag' column should be applied to your LE and H variables:

```bash
latent_heat_flux_qc = QC_flag
sensible_heat_flux_qc = QC_flag
```

Another option is to use a numeric quality control *value* that exists in your input data along with a threshold value which means that when the quality control value falls below this threshold you would like to exclude it from the analysis. Let's assume the column containing the quality control values is named 'QC_values' and it contains values between 0 and 1 with 0 meaning the poorest quality data and 1 being the highest and that you would like to remove all data for select variables with a quality control value below 0.5. Let's further assume that you would like this to apply to your incoming solar radiation variable. Then you would declare the threshold in the **METADATA** section of your config file:

```bash
qc_threshold = 0.5
```

and in the **DATA** section of your config you will state that the 'QC_value' column should be applied to your incoming shortwave radiation variable:

```bash
shortwave_in_qc = QC_value
```

Now you are all set to use the functionality, note that you may apply the same quality control value or flag column to multiple climate variables (as shown in the first example). You may also use both numeric qualtiy control values and character string flags for the same input dataset although they cannot both be applied to the same variable. In other wordsf, if you have a column of quality control numeric values it cannot also have character strings mixed in. Another option that is used in the example below is to declare multiple quality control flags that should be filtered out using a comma separated list. For example in the provided example config the flags 'x' and 'b' are used to remove select days from incoming shorwave radiation,

```bash
qc_flag = x, b
```

There is another option for specifying variables quality control values/flags. Name the column containing the qualtiy control value/flag in your input climate file the same as the variable it corresponds to with the suffix **'_QC'**. For example if your sensible heat column was named **sens_h** then your qualtiy control column should be named **sens_h_QC**. This is the case for FLUXNET data. If you use this option you do not need to specify the names in your config file. Below is an example using the provided FLUXNET file which includes its own qualtiy control flags for sensible heat and others. Note that if your data's qualtiy controlheader names follow this convention they will automatically be detected and used when you apply them using `Data.apply_qc_flags`.

In [3]:
config_path = 'multiple_soilflux_config.ini'
d = Data(config_path)
# view or reassign the numeric threshold specified in the config file
d.qc_threshold

0.5

In [4]:
# view the list of string flags specified in the config file
d.qc_flag

['x', 'b']

In [5]:
# this attribute shows you which variables were found in your input file that have
# quality control values assigned, it uses the names as found in the input file
d.qc_var_pairs

{'LE': 'a_qc_value', 'H': 'a_qc_value', 'sw_in': 'swrad_flag'}

# Apply QC values and flags to select variables and view results

Note that in this example we mixed both numeric values and threshold with character flags, the numeric values are being applied to LE and H whereas the flags ('x' and 'b') are applied to incoming shorwave radiation.

In [6]:
# make copys of before and after the QC filter is applied
no_qc = d.df.LE.copy()
no_qc_swrad = d.df.sw_in.copy()
# apply QC flags/values
d.apply_qc_flags()
qc_def = d.df.LE.copy()
qc_flag_swrad = d.df.sw_in.copy()

g weights do not sum to one, normalizing
Here are the new weights:
 added_G_col:0.71, another_G_var:0.24, G:0.06


# View results of filter before and after

In [7]:
p = figure(x_axis_label='date', y_axis_label='swrad with data removed based on QC value')
p.line(no_qc_swrad.index, no_qc_swrad, color='red', legend="no flag", line_width=2)
p.line(no_qc_swrad.index, qc_flag_swrad, color='black', legend="flag = b or x", line_width=2)
p.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%Y")
show(p)

In [8]:
# for LE
p = figure(x_axis_label='date', y_axis_label='LE with data removed based on QC value')
p.line(no_qc.index, no_qc, color='red', legend="no QC", line_width=2)
p.line(no_qc.index, qc_def, color='black', legend="QC=0.5", line_width=2)
p.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%Y")
show(p)

# Another example where column names for QC columns do not need to be explicitly declared in the config file

In this case the climate variables QC columns are named with the same base name as the climate variables with the '_QC' suffix. For example if LE is named 'LE_F_MDS' in your input files header then the QC column is named 'LE_F_MDS_QC'.

In [9]:
config_path = 'fluxnet_config.ini'
d = Data(config_path)
# view input files header, note the QC columns 
d.header

Index(['TIMESTAMP', 'TA_F', 'TA_F_QC', 'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC',
       'LW_IN_F', 'LW_IN_F_QC', 'VPD_F', 'VPD_F_QC', 'PA_F', 'PA_F_QC', 'P_F',
       'P_F_QC', 'WS_F', 'WS_F_QC', 'USTAR', 'USTAR_QC', 'NETRAD', 'NETRAD_QC',
       'PPFD_IN', 'PPFD_IN_QC', 'PPFD_OUT', 'PPFD_OUT_QC', 'SW_OUT',
       'SW_OUT_QC', 'LW_OUT', 'LW_OUT_QC', 'CO2_F_MDS', 'CO2_F_MDS_QC',
       'TS_F_MDS_1', 'TS_F_MDS_1_QC', 'SWC_F_MDS_1', 'SWC_F_MDS_1_QC',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'LE_CORR_25', 'LE_CORR_75', 'LE_RANDUNC', 'H_F_MDS', 'H_F_MDS_QC',
       'H_CORR', 'H_CORR_25', 'H_CORR_75', 'H_RANDUNC', 'NEE_VUT_REF',
       'NEE_VUT_REF_QC', 'NEE_VUT_REF_RANDUNC', 'NEE_VUT_25', 'NEE_VUT_50',
       'NEE_VUT_75', 'NEE_VUT_25_QC', 'NEE_VUT_50_QC', 'NEE_VUT_75_QC',
       'RECO_NT_VUT_REF', 'RECO_NT_VUT_25', 'RECO_NT_VUT_50', 'RECO_NT_VUT_75',
       'GPP_NT_VUT_REF', 'GPP_NT_VUT_25', 'GPP_NT_VUT_50', 'GPP_NT_VUT_75',
       'RECO_DT_VUT_REF', 'RECO_D

In [10]:
# verify that the QC columns have been paired with corresponding climate variables
d.qc_var_pairs

{'NETRAD': 'NETRAD_QC',
 'G_F_MDS': 'G_F_MDS_QC',
 'LE_F_MDS': 'LE_F_MDS_QC',
 'H_F_MDS': 'H_F_MDS_QC',
 'SW_IN_F': 'SW_IN_F_QC',
 'SW_OUT': 'SW_OUT_QC',
 'LW_IN_F': 'LW_IN_F_QC',
 'LW_OUT': 'LW_OUT_QC',
 'VPD_F': 'VPD_F_QC',
 'TA_F': 'TA_F_QC',
 'P_F': 'P_F_QC',
 'WS_F': 'WS_F_QC'}

## Note that for this dataset we did not specify a QC threshold or flag(s) in the config

We can assign it when calling the `Data.apply_qc_flags` method.

In [11]:
# view the QC threshold specified in the config file
print(d.qc_threshold, type(d.qc_threshold))

None <class 'NoneType'>


# Visualize the QC values alongside corresponding data

If you create your own QC values be sure to validate them to make sure everything seems correct. Below we see that the lowest QC values correspond with poor quality gap-fill data near the begining of the dataset. 

In [12]:
p = figure(x_axis_label='date', y_axis_label='sensible heat flux (w/m2)')
p.extra_y_ranges = {"sec": Range1d(start=-0.1, end=1.1)}
p.line(d.df.index, d.df['H_F_MDS'], color='red', line_width=1, legend='data')
p.add_layout(LinearAxis(y_range_name="sec", axis_label='QC value'), 'right')
p.circle(d.df.index, d.df['H_F_MDS_QC'], line_width=2, y_range_name="sec", legend='QC')
p.x_range=Range1d(d.df.index[0], d.df.index[365])
p.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%Y")
p.legend.location = "top_left"
show(p)

The routine provided removes all data that falls below a QC value of 0.5. Although this can be modified, see the [provided FLUXNET Jupyter notebook](https://github.com/Open-ET/flux-data-qaqc/blob/master/examples/FLUXNET_2015_example.ipynb) for examples.

# After applying QC filter removing values below 0.5

Values of sensible heat with QC values < 0.5 are now removed (null). 

Note, the `Data.apply_qc_flags()` method applies the filter to all variables in the climate file that have a QC column.  

In [13]:
# apply QC filters
d.apply_qc_flags(threshold=0.5)
# same figure
p = figure(x_axis_label='date', y_axis_label='sensible heat flux (w/m2)')
p.extra_y_ranges = {"sec": Range1d(start=-0.1, end=1.1)}
p.line(d.df.index, d.df['H_F_MDS'], color='red', line_width=1, legend='data')
p.add_layout(LinearAxis(y_range_name="sec", axis_label='QC value'), 'right')
p.circle(d.df.index, d.df['H_F_MDS_QC'], line_width=2, y_range_name="sec", legend='QC')
p.x_range=Range1d(d.df.index[0], d.df.index[365])
p.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%Y")
p.legend.location = "top_left"
show(p)

---

# Specifying multiples soil heat flux and soil moisture variables for your input

`flux-data-qaqc` provides the ability to read in multiple soil heat flux/moisture variables for a given station location, calculate their weighted or non weighted average, and write/plot their daily and monthly time series. This may be useful for comparing/validating multiple soil heat/moisture probes at varying locations or depths or varying instrumentation. 

Here is what you need to do to use this functionality:

1. List the multiple soil variable names in your config file following the convention:
 * For multiple soil heat flux variables config names should begin with "G_" or "g_" followed by an integer starting with 1,2,3,... i.e. g_[number]. For example:

```bash
g_1 = name_of_my_soil_heat_flux_variable
```
 * For soil moisture variables the name of the config variable should follow "theta_[number]" for example:

```bash
theta_1 = name_of_my_soil_moisture_variable
```

2. List the units and (optionally) weights of the multiple variables 
 * To specify the units of your soil flux/moisture variables add "_units" to the config name you assigned:

```bash
g_1_units = w/m2
theta_1_units = cm
```

 * To set weights for multiple variables to compute weighted averages assign the "_weight" suffix to their names in the config file. For example, to set weights for multiple soil heat flux variables:
```bash
g_1_weight = 0.25
g_2_weight = 0.25
g_3_weight = 0.5
```
 * Note, if weights are not given the arithmetic mean will be calculated, if the weights do not sum to 1, they will be automatically normalized so that they do. 

# Multiple soil variable weighted average examples

The provided multiple soil variable config and input data are used for these examples.

Here is the section of the config file that defines the multiple soil variables in the input climate file used for the example below:

```bash
g_1 = added_G_col
g_1_weight = 6
g_1_units = w/m2
g_2 = another_G_var
g_2_weight = 2
g_2_units = w/m2
# note the next variable is the same that was assigned as the main soil heat flux variable
# i.e. ground_flux_col = G
g_3 = G
g_3_weight = 0.5
g_3_units = w/m2

theta_1 = soil_moisture_z1
theta_1_weight = 0.25
theta_1_units = cm
theta_2 = soil_moisture_z10
theta_2_weight = 0.75
theta_2_units = cm
```

In [14]:
# read in the data
config_path = 'multiple_soilflux_config.ini'
d = Data(config_path)
# note the newly added multiple g and theata variables
d.variables

{'date': 'date',
 'year': 'na',
 'month': 'na',
 'day': 'na',
 'Rn': 'Rn',
 'G': 'G',
 'LE': 'LE',
 'LE_user_corr': 'LE_corrected',
 'H': 'H',
 'H_user_corr': 'H_corrected',
 'sw_in': 'sw_in',
 'sw_out': 'sw_out',
 'sw_pot': 'sw_pot',
 'lw_in': 'lw_in',
 'lw_out': 'lw_out',
 'vp': 'na',
 'vpd': 'vpd',
 't_avg': 't_avg',
 'ppt': 'ppt',
 'ws': 'ws',
 'g_1': 'added_G_col',
 'g_2': 'another_G_var',
 'g_3': 'G',
 'theta_1': 'soil_moisture_z1',
 'theta_2': 'soil_moisture_z10',
 'LE_qc_flag': 'a_qc_value',
 'H_qc_flag': 'a_qc_value',
 'sw_in_qc_flag': 'swrad_flag'}

In [15]:
# and their units
d.units

{'Rn': 'w/m2',
 'G': 'w/m2',
 'LE': 'w/m2',
 'LE_user_corr': 'w/m2',
 'H': 'w/m2',
 'H_user_corr': 'w/m2',
 'sw_in': 'w/m2',
 'sw_out': 'w/m2',
 'sw_pot': 'w/m2',
 'lw_in': 'w/m2',
 'lw_out': 'w/m2',
 'vp': 'na',
 'vpd': 'hPa',
 't_avg': 'C',
 'ppt': 'mm',
 'ws': 'm/s',
 'g_1': 'w/m2',
 'g_2': 'w/m2',
 'theta_1': 'cm',
 'theta_2': 'cm'}

In [16]:
# or to view these variables and their weights only
d.soil_var_weight_pairs

{'g_1': {'name': 'added_G_col', 'weight': '6'},
 'g_2': {'name': 'another_G_var', 'weight': '2'},
 'g_3': {'name': 'G', 'weight': '0.5'},
 'theta_1': {'name': 'soil_moisture_z1', 'weight': '0.25'},
 'theta_2': {'name': 'soil_moisture_z10', 'weight': '0.75'}}

# When the data is first loaded into memory the weighted averages are calculated

At this stage weights will be automatically normalized so that they sum to one and the new weights will be printed if this occurs.

In [17]:
# call daily or monthly dataframe to calculate the weighted averages if they exist
d.df.head()

g weights do not sum to one, normalizing
Here are the new weights:
 added_G_col:0.71, another_G_var:0.24, G:0.06


,t_avg,sw_pot,sw_in,lw_in,vpd,ppt,ws,Rn,sw_out,lw_out,...,H,H_corrected,added_G_col,another_G_var,soil_moisture_z1,soil_moisture_z10,a_qc_value,swrad_flag,g_mean,theta_mean
date,,,,,,,,,,,,,,,,,,,,,
2009-01-01,2.803,186.710,123.108,261.302,1.919,0.0,3.143,NaN,NaN,NaN,...,20.3876,13.3116,NaN,NaN,20.573270,26.942860,0,x,0.0,25.350463
2009-01-02,2.518,187.329,121.842,268.946,0.992,0.0,2.093,NaN,NaN,NaN,...,32.6505,21.4364,NaN,NaN,20.250870,26.601709,0,x,0.0,25.013999
2009-01-03,5.518,188.008,124.241,268.004,2.795,0.0,4.403,NaN,NaN,NaN,...,20.0569,13.3130,NaN,NaN,20.827236,26.644598,0,x,0.0,25.190258
2009-01-04,-3.753,188.742,113.793,246.675,0.892,0.0,4.336,NaN,NaN,NaN,...,20.3876,13.6798,NaN,NaN,20.988757,26.843588,0,x,0.0,25.379880
2009-01-05,-2.214,189.534,124.332,244.478,1.304,0.0,2.417,NaN,NaN,NaN,...,32.6505,22.0260,NaN,NaN,20.756527,26.262146,0,x,0.0,24.885741


In [18]:
# note the weights have been changed and updated 
d.soil_var_weight_pairs

{'g_1': {'name': 'added_G_col', 'weight': 0.7058823529411765},
 'g_2': {'name': 'another_G_var', 'weight': 0.23529411764705882},
 'g_3': {'name': 'G', 'weight': 0.058823529411764705},
 'theta_1': {'name': 'soil_moisture_z1', 'weight': '0.25'},
 'theta_2': {'name': 'soil_moisture_z10', 'weight': '0.75'}}

In [19]:
# now the dataframe also has the weighted means that will be named g_mean and theta_mean
d.df.columns

Index(['t_avg', 'sw_pot', 'sw_in', 'lw_in', 'vpd', 'ppt', 'ws', 'Rn', 'sw_out',
       'lw_out', 'G', 'LE', 'LE_corrected', 'H', 'H_corrected', 'added_G_col',
       'another_G_var', 'soil_moisture_z1', 'soil_moisture_z10', 'a_qc_value',
       'swrad_flag', 'g_mean', 'theta_mean'],
      dtype='object')

###  The weighted mean is closest to the variable assigned to "g_1" which had the highest weight

In [20]:
p = figure(x_axis_label='date', y_axis_label='Soil heat flux')
p.line(d.df.index, d.df['g_mean'], color='black', legend="weighted mean", line_width=2)
p.line(d.df.index, d.df['added_G_col'], color='orange', legend="g_1: 0.71", line_width=1)
p.line(d.df.index, d.df['another_G_var'], color='green', legend="g_2: 0.24", line_width=1)
p.line(d.df.index, d.df['G'], color='red', legend="g_3: 0.60", line_width=1)

p.x_range=Range1d(d.df.index[150], d.df.index[160])
p.xaxis.formatter = DatetimeTickFormatter(days="%d-%b-%Y")
show(p)